In [4]:
import pandas as pd

input_csv = "../data/raw/ksa_regions_cities_districts/GeoAdministrativeUnits.csv"
df = pd.read_csv(input_csv)

print("Shape:", df.shape)
print("\nCOLUMNS:")
for i, c in enumerate(df.columns, 1):
    print(f"{i:02d}. {c}")

df.head()

Shape: (3732, 6)

COLUMNS:
01. district_name_ar
02. district_name_en
03. city_name_ar
04. region_name_ar
05. latitude
06. longitude


,district_name_ar,district_name_en,city_name_ar,region_name_ar,latitude,longitude
0,حي الصفا,Al Safa Dist.,تبوك,منطقة تبوك,28.375892,36.512862
1,حي الاخضر,Al Akhdar Dist.,تبوك,منطقة تبوك,28.361020,36.510876
2,حي البوادي,Al Bawadi Dist.,تبوك,منطقة تبوك,28.361720,36.524985
3,حي اليرموك,Al Yarmuk Dist.,تبوك,منطقة تبوك,28.352558,36.513125
4,الأحياء الجنوبية,Southern Districts,تبوك,منطقة تبوك,28.373269,36.544050


In [5]:
# Cleaning Step 1

df_clean = df.copy()

# 1) تنظيف الأعمدة النصية
text_cols = df_clean.select_dtypes(include="object").columns
print("Text columns:", list(text_cols))

for c in text_cols:
    df_clean[c] = df_clean[c].astype(str).str.strip()

# 2) حذف الصفوف الفاضية بالكامل
before = df_clean.shape[0]
df_clean = df_clean.dropna(how="all")
after = df_clean.shape[0]
print("Dropped empty rows:", before - after)

# 3) حذف التكرارات الكاملة
before = df_clean.shape[0]
df_clean = df_clean.drop_duplicates()
after = df_clean.shape[0]
print("Dropped duplicate rows:", before - after)

df_clean.head()


Text columns: ['district_name_ar', 'district_name_en', 'city_name_ar', 'region_name_ar']
Dropped empty rows: 0
Dropped duplicate rows: 7


,district_name_ar,district_name_en,city_name_ar,region_name_ar,latitude,longitude
0,حي الصفا,Al Safa Dist.,تبوك,منطقة تبوك,28.375892,36.512862
1,حي الاخضر,Al Akhdar Dist.,تبوك,منطقة تبوك,28.361020,36.510876
2,حي البوادي,Al Bawadi Dist.,تبوك,منطقة تبوك,28.361720,36.524985
3,حي اليرموك,Al Yarmuk Dist.,تبوك,منطقة تبوك,28.352558,36.513125
4,الأحياء الجنوبية,Southern Districts,تبوك,منطقة تبوك,28.373269,36.544050


In [6]:
# Normalization Step 1 (text normalization only)

import re

def normalize_text(series):
    s = series.astype(str)

    # إزالة علامات اتجاه النص اللي تظهر أحيانًا بالعربي
    s = s.str.replace("\u200f", "", regex=False)  # RTL
    s = s.str.replace("\u200e", "", regex=False)  # LTR

    # lower + strip
    s = s.str.strip().str.lower()

    # توحيد المسافات
    s = s.str.replace(r"\s+", " ", regex=True)

    return s

df_norm = df_clean.copy()

for c in text_cols:
    df_norm[c + "_norm"] = normalize_text(df_norm[c])

print("Added normalized columns:")
for c in df_norm.columns:
    if c.endswith("_norm"):
        print("-", c)

df_norm.head()

Added normalized columns:
- district_name_ar_norm
- district_name_en_norm
- city_name_ar_norm
- region_name_ar_norm


,district_name_ar,district_name_en,city_name_ar,region_name_ar,latitude,longitude,district_name_ar_norm,district_name_en_norm,city_name_ar_norm,region_name_ar_norm
0,حي الصفا,Al Safa Dist.,تبوك,منطقة تبوك,28.375892,36.512862,حي الصفا,al safa dist.,تبوك,منطقة تبوك
1,حي الاخضر,Al Akhdar Dist.,تبوك,منطقة تبوك,28.361020,36.510876,حي الاخضر,al akhdar dist.,تبوك,منطقة تبوك
2,حي البوادي,Al Bawadi Dist.,تبوك,منطقة تبوك,28.361720,36.524985,حي البوادي,al bawadi dist.,تبوك,منطقة تبوك
3,حي اليرموك,Al Yarmuk Dist.,تبوك,منطقة تبوك,28.352558,36.513125,حي اليرموك,al yarmuk dist.,تبوك,منطقة تبوك
4,الأحياء الجنوبية,Southern Districts,تبوك,منطقة تبوك,28.373269,36.544050,الأحياء الجنوبية,southern districts,تبوك,منطقة تبوك


In [7]:
import pandas as pd

# 4-cities mapping (Arabic normalized -> geo key + id)
CITY_MAP = {
    "الرياض": ("riyadh", 1),
    "جدة": ("jeddah", 2),
    "الخبر": ("khobar", 3),
    "الدمام": ("dammam", 4),
}

def to_city_geo_id(city_ar_norm: str):
    if not isinstance(city_ar_norm, str):
        return (None, None)
    return CITY_MAP.get(city_ar_norm, (city_ar_norm, None))  # باقي المدن: نخليها نص + id = None

df_norm[["city_geo", "city_id"]] = (
    df_norm["city_name_ar_norm"]
    .apply(to_city_geo_id)
    .apply(pd.Series)
)

df_norm["city_id"] = df_norm["city_id"].astype("Int64")  # أرقام + يسمح بـ NA

# عرض سريع للتأكد
df_norm[["city_name_ar", "city_name_ar_norm", "city_geo", "city_id"]].head(10)

,city_name_ar,city_name_ar_norm,city_geo,city_id
0,تبوك,تبوك,تبوك,<NA>
1,تبوك,تبوك,تبوك,<NA>
2,تبوك,تبوك,تبوك,<NA>
3,تبوك,تبوك,تبوك,<NA>
4,تبوك,تبوك,تبوك,<NA>
5,تبوك,تبوك,تبوك,<NA>
6,تبوك,تبوك,تبوك,<NA>
7,تبوك,تبوك,تبوك,<NA>
8,تبوك,تبوك,تبوك,<NA>
9,تبوك,تبوك,تبوك,<NA>


In [8]:
df_norm["city_id"].value_counts(dropna=False)

city_id
<NA>    3238
1        189
2        167
4         87
3         44
Name: count, dtype: Int64

In [10]:
import os
import pandas as pd

# إنشاء المسار
out_dir = "../data/processed/geo"
os.makedirs(out_dir, exist_ok=True)

# =========================
# 1) geo_cities.parquet (قبل الفلترة)
# =========================
geo_cities_all = (
    df_norm
    .groupby(["city_geo", "city_id", "city_name_ar", "region_name_ar"], dropna=False)
    .agg(
        latitude=("latitude", "mean"),
        longitude=("longitude", "mean"),
    )
    .reset_index()
    .sort_values(["city_id", "city_geo"], na_position="last")
)

geo_cities_all.to_parquet(
    f"{out_dir}/geo_cities.parquet",
    index=False
)

# =========================
# 2) districts.parquet (قبل الفلترة)
# =========================
districts_all = df_norm[
    [
        "district_name_ar",
        "district_name_en",
        "district_name_ar_norm",
        "district_name_en_norm",
        "city_geo",
        "city_id",
        "latitude",
        "longitude",
    ]
].copy()

districts_all.to_parquet(
    f"{out_dir}/districts.parquet",
    index=False
)

# =========================
# 3) city_aliases.csv (لو ما كان محفوظ)
# =========================
aliases_path = f"{out_dir}/city_aliases.csv"
if not os.path.exists(aliases_path):
    print("⚠️ city_aliases.csv غير موجود – احفظيه من خلية المابنق")
else:
    print("✅ city_aliases.csv موجود")

print("Saved (pre-filter) files:")
print("-", f"{out_dir}/geo_cities.parquet")
print("-", f"{out_dir}/districts.parquet")

⚠️ city_aliases.csv غير موجود – احفظيه من خلية المابنق
Saved (pre-filter) files:
- ../data/processed/geo/geo_cities.parquet
- ../data/processed/geo/districts.parquet


In [11]:
import pandas as pd
import os

out_dir = "../data/processed/geo"
os.makedirs(out_dir, exist_ok=True)

city_aliases = pd.DataFrame([
    # Riyadh
    {"alias": "الرياض", "city_geo": "riyadh", "city_id": 1},
    {"alias": "riyadh", "city_geo": "riyadh", "city_id": 1},
    {"alias": "ar riyadh", "city_geo": "riyadh", "city_id": 1},
    {"alias": "al riyadh", "city_geo": "riyadh", "city_id": 1},

    # Jeddah
    {"alias": "جدة", "city_geo": "jeddah", "city_id": 2},
    {"alias": "jeddah", "city_geo": "jeddah", "city_id": 2},
    {"alias": "jidda", "city_geo": "jeddah", "city_id": 2},

    # Khobar
    {"alias": "الخبر", "city_geo": "khobar", "city_id": 3},
    {"alias": "khobar", "city_geo": "khobar", "city_id": 3},
    {"alias": "al khobar", "city_geo": "khobar", "city_id": 3},

    # Dammam
    {"alias": "الدمام", "city_geo": "dammam", "city_id": 4},
    {"alias": "dammam", "city_geo": "dammam", "city_id": 4},
])

# normalization للـ alias
city_aliases["alias_norm"] = (
    city_aliases["alias"]
    .astype(str)
    .str.strip()
    .str.lower()
    .str.replace(r"\s+", " ", regex=True)
)

city_aliases = city_aliases.drop_duplicates()

out_path = f"{out_dir}/city_aliases.csv"
city_aliases.to_csv(out_path, index=False, encoding="utf-8-sig")

print("Saved:", out_path)
city_aliases

Saved: ../data/processed/geo/city_aliases.csv


,alias,city_geo,city_id,alias_norm
0,الرياض,riyadh,1,الرياض
1,riyadh,riyadh,1,riyadh
2,ar riyadh,riyadh,1,ar riyadh
3,al riyadh,riyadh,1,al riyadh
4,جدة,jeddah,2,جدة
5,jeddah,jeddah,2,jeddah
6,jidda,jeddah,2,jidda
7,الخبر,khobar,3,الخبر
8,khobar,khobar,3,khobar
9,al khobar,khobar,3,al khobar


In [12]:
import os
import pandas as pd

# إنشاء مجلد final
final_dir = "../data/final"
os.makedirs(final_dir, exist_ok=True)

# =========================
# 1) فلترة المدن المختارة
# =========================
df_final = df_norm[df_norm["city_id"].isin([1, 2, 3, 4])].copy()
print("Final shape:", df_final.shape)

# =========================
# 2) selected_cities.csv
# =========================
selected_cities = (
    df_final[["city_id", "city_geo", "city_name_ar", "region_name_ar"]]
    .drop_duplicates()
    .sort_values("city_id")
)

selected_cities.to_csv(
    f"{final_dir}/selected_cities.csv",
    index=False,
    encoding="utf-8-sig"
)

# =========================
# 3) districts_selected.parquet (لو احتجتيه لاحقًا)
# =========================
districts_selected = df_final[
    [
        "district_name_ar",
        "district_name_en",
        "district_name_ar_norm",
        "district_name_en_norm",
        "city_id",
        "city_geo",
        "latitude",
        "longitude",
    ]
].copy()

districts_selected.to_parquet(
    f"{final_dir}/districts_selected.parquet",
    index=False
)

print("Saved in final/:")
print("-", f"{final_dir}/selected_cities.csv")
print("-", f"{final_dir}/districts_selected.parquet")

selected_cities


Final shape: (487, 12)
Saved in final/:
- ../data/final/selected_cities.csv
- ../data/final/districts_selected.parquet


,city_id,city_geo,city_name_ar,region_name_ar
64,1,riyadh,الرياض,منطقة الرياض
914,2,jeddah,جدة,منطقة مكة المكرمة
1097,3,khobar,الخبر,المنطقة الشرقية
650,4,dammam,الدمام,المنطقة الشرقية


In [13]:
import pandas as pd

df = pd.read_parquet("../data/final/districts_selected.parquet")
df.head()

,district_name_ar,district_name_en,district_name_ar_norm,district_name_en_norm,city_id,city_geo,latitude,longitude
0,حي العمل,Al Amal Dist.,حي العمل,al amal dist.,1,riyadh,24.645433,46.719294
1,حي النموذجية,Al Namudhajiyah Dist.,حي النموذجية,al namudhajiyah dist.,1,riyadh,24.658362,46.695167
2,حي الجرادية,Al Jarradiyah Dist.,حي الجرادية,al jarradiyah dist.,1,riyadh,24.616732,46.692920
3,حي الفلاح,Al Falah Dist.,حي الفلاح,al falah dist.,1,riyadh,24.796231,46.706683
4,حي الندى,Al Nada Dist.,حي الندى,al nada dist.,1,riyadh,24.800200,46.679995


In [14]:
import os
import shutil
import kagglehub

DATASET = "shahlaliaquat/2024-solar-dataset-of-top-5-cities-of-ksa"

# 1) تنزيل (يخزن في cache)
path = kagglehub.dataset_download(DATASET)
print("Downloaded to cache:", path)

# 2) نسخ إلى مشروعك داخل raw
target = "../data/raw/solar_top5_cities_2024"
os.makedirs(target, exist_ok=True)

# انسخ كل الملفات (لو موجودة من قبل ما يخرب)
for item in os.listdir(path):
    src = os.path.join(path, item)
    dst = os.path.join(target, item)
    if os.path.isdir(src):
        shutil.copytree(src, dst, dirs_exist_ok=True)
    else:
        shutil.copy2(src, dst)

print("Copied to project:", target)
print("Files:", os.listdir(target))

c:\Users\itsha\Desktop\DarbakGP\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 35.5k/35.5k [00:00<00:00, 718kB/s]

Extracting files...
Downloaded to cache: C:\Users\itsha\.cache\kagglehub\datasets\shahlaliaquat\2024-solar-dataset-of-top-5-cities-of-ksa\versions\1
Copied to project: ../data/raw/solar_top5_cities_2024
Files: ['ksa_solar_dataset_2024_detailed.csv']


In [16]:
import pandas as pd

input_csv = "../data/raw/solar_top5_cities_2024/ksa_solar_dataset_2024_detailed.csv"

solar = pd.read_csv(input_csv, encoding="latin1")

print("Shape:", solar.shape)
print("\nCOLUMNS:")
for i, c in enumerate(solar.columns, 1):
    print(f"{i:02d}. {c}")

solar.head()

Shape: (10980, 13)

COLUMNS:
01. Date
02. City
03. Latitude
04. Longitude
05. Temperature (°C)
06. Weather
07. Panel Type
08. Mount Type
09. Tilt (°)
10. Panel Efficiency (%)
11. Solar Irradiance (kWh/m²/day)
12. Output Power (W)
13. Estimated Daily Output (kWh)


,Date,City,Latitude,Longitude,Temperature (°C),Weather,Panel Type,Mount Type,Tilt (°),Panel Efficiency (%),Solar Irradiance (kWh/m²/day),Output Power (W),Estimated Daily Output (kWh)
0,1/1/2024,Riyadh,24.7136,46.6753,18,few clouds,Monocrystalline,Rooftop,25,22,5.39,1.90,21.06
1,1/1/2024,Riyadh,24.7136,46.6753,18,few clouds,Polycrystalline,Rooftop,25,18,5.39,1.55,21.06
2,1/1/2024,Riyadh,24.7136,46.6753,18,few clouds,Thin-Film,Rooftop,25,12,5.39,1.04,21.06
3,1/1/2024,Riyadh,24.7136,46.6753,18,few clouds,Monocrystalline,Ground,15,22,4.97,1.75,19.47
4,1/1/2024,Riyadh,24.7136,46.6753,18,few clouds,Polycrystalline,Ground,15,18,4.97,1.43,19.47


In [17]:
import pandas as pd

input_csv = "../data/raw/solar_top5_cities_2024/ksa_solar_dataset_2024_detailed.csv"

solar = pd.read_csv(input_csv, encoding="latin1")

print("Shape:", solar.shape)
print("\nCOLUMNS:")
for i, c in enumerate(solar.columns, 1):
    print(f"{i:02d}. {c}")

solar.head()

Shape: (10980, 13)

COLUMNS:
01. Date
02. City
03. Latitude
04. Longitude
05. Temperature (°C)
06. Weather
07. Panel Type
08. Mount Type
09. Tilt (°)
10. Panel Efficiency (%)
11. Solar Irradiance (kWh/m²/day)
12. Output Power (W)
13. Estimated Daily Output (kWh)


,Date,City,Latitude,Longitude,Temperature (°C),Weather,Panel Type,Mount Type,Tilt (°),Panel Efficiency (%),Solar Irradiance (kWh/m²/day),Output Power (W),Estimated Daily Output (kWh)
0,1/1/2024,Riyadh,24.7136,46.6753,18,few clouds,Monocrystalline,Rooftop,25,22,5.39,1.90,21.06
1,1/1/2024,Riyadh,24.7136,46.6753,18,few clouds,Polycrystalline,Rooftop,25,18,5.39,1.55,21.06
2,1/1/2024,Riyadh,24.7136,46.6753,18,few clouds,Thin-Film,Rooftop,25,12,5.39,1.04,21.06
3,1/1/2024,Riyadh,24.7136,46.6753,18,few clouds,Monocrystalline,Ground,15,22,4.97,1.75,19.47
4,1/1/2024,Riyadh,24.7136,46.6753,18,few clouds,Polycrystalline,Ground,15,18,4.97,1.43,19.47


In [18]:
# Step: Rename columns to clean snake_case names (no value changes)

solar = solar.rename(columns={
    "Date": "date",
    "City": "city_raw",
    "Latitude": "latitude",
    "Longitude": "longitude",
    "Temperature (°C)": "temperature_c",
    "Weather": "weather",
    "Panel Type": "panel_type",
    "Mount Type": "mount_type",
    "Tilt (°)": "tilt_deg",
    "Panel Efficiency (%)": "panel_efficiency_pct",
    "Solar Irradiance (kWh/m²/day)": "solar_irradiance_kwh_m2_day",
    "Output Power (W)": "output_power_w",
    "Estimated Daily Output (kWh)": "estimated_daily_output_kwh",
})

# quick check
solar.columns

Index(['date', 'city_raw', 'latitude', 'longitude', 'temperature_c', 'weather',
       'panel_type', 'mount_type', 'tilt_deg', 'panel_efficiency_pct',
       'solar_irradiance_kwh_m2_day', 'output_power_w',
       'estimated_daily_output_kwh'],
      dtype='object')

In [19]:
# Step 2: Normalize city name and map to city_id (no filtering yet)

import re

# 1) Normalize city names in solar dataset
def normalize_city_name(series):
    s = series.astype(str)
    s = s.str.strip().str.lower()
    s = s.str.replace(r"\s+", " ", regex=True)
    return s

solar["city_norm"] = normalize_city_name(solar["city_raw"])

# 2) Load geo_cities reference
geo_cities = pd.read_parquet("../data/processed/geo/geo_cities.parquet")

# تأكد إن city_geo جاهز
geo_cities["city_geo"] = normalize_city_name(geo_cities["city_geo"])

# 3) Build mapping: city_geo → city_id
city_id_map = (
    geo_cities
    .dropna(subset=["city_id"])
    .drop_duplicates("city_geo")
    .set_index("city_geo")["city_id"]
)

# 4) Map city_id into solar dataset
solar["city_id"] = solar["city_norm"].map(city_id_map)

# quick checks
solar[["city_raw", "city_norm", "city_id"]].head()

,city_raw,city_norm,city_id
0,Riyadh,riyadh,1
1,Riyadh,riyadh,1
2,Riyadh,riyadh,1
3,Riyadh,riyadh,1
4,Riyadh,riyadh,1


In [20]:
solar[["city_raw", "city_norm", "city_id"]].drop_duplicates()

,city_raw,city_norm,city_id
0,Riyadh,riyadh,1
2196,Jeddah,jeddah,2
4392,Mecca,mecca,<NA>
6588,Medina,medina,<NA>
8784,Dammam,dammam,4


In [21]:
# Step: Restrict to project cities only + correct city_id mapping

city_id_map = {
    "riyadh": 1,
    "jeddah": 2,
    "khobar": 3,
    "dammam": 4,
}

# نحتفظ فقط بمدن المشروع
solar = solar[solar["city_norm"].isin(city_id_map.keys())].copy()

# نربط city_id بشكل صريح
solar["city_id"] = solar["city_norm"].map(city_id_map).astype("int64")

# تحقق نهائي
solar[["city_raw", "city_norm", "city_id"]].drop_duplicates()

,city_raw,city_norm,city_id
0,Riyadh,riyadh,1
2196,Jeddah,jeddah,2
8784,Dammam,dammam,4


In [22]:
# Step: Type cleaning (date + numeric columns)

import pandas as pd

# 1) date -> datetime (الملف عندك بصيغة m/d/yyyy)
solar["date"] = pd.to_datetime(solar["date"], errors="coerce")

# 2) الأعمدة الرقمية -> numeric
num_cols = [
    "latitude",
    "longitude",
    "temperature_c",
    "tilt_deg",
    "panel_efficiency_pct",
    "solar_irradiance_kwh_m2_day",
    "output_power_w",
    "estimated_daily_output_kwh",
]

for c in num_cols:
    solar[c] = pd.to_numeric(solar[c], errors="coerce")

# 3) فحص سريع: كم قيمة فاضية صارت بعد التحويل؟
print("Nulls check:")
print(solar[["date"] + num_cols].isna().sum().sort_values(ascending=False).head(10))

solar.dtypes

Nulls check:
date                           0
latitude                       0
longitude                      0
temperature_c                  0
tilt_deg                       0
panel_efficiency_pct           0
solar_irradiance_kwh_m2_day    0
output_power_w                 0
estimated_daily_output_kwh     0
dtype: int64


date                           datetime64[ns]
city_raw                               object
latitude                              float64
longitude                             float64
temperature_c                           int64
weather                                object
panel_type                             object
mount_type                             object
tilt_deg                                int64
panel_efficiency_pct                    int64
solar_irradiance_kwh_m2_day           float64
output_power_w                        float64
estimated_daily_output_kwh            float64
city_norm                              object
city_id                                 int64
dtype: object

In [23]:
# Step: Final text cleaning + deduplication + save to processed/weather

import os

# 1) text normalization (خفيف – بدون تغيير معنى)
text_cols = ["weather", "panel_type", "mount_type"]

for c in text_cols:
    solar[c] = (
        solar[c]
        .astype(str)
        .str.strip()
        .str.lower()
        .str.replace(r"\s+", " ", regex=True)
    )

# 2) remove exact duplicates
before = len(solar)
solar = solar.drop_duplicates()
after = len(solar)

print(f"Removed duplicates: {before - after}")

# 3) ensure output directory exists
out_dir = "../data/processed/weather"
os.makedirs(out_dir, exist_ok=True)

# 4) save processed solar dataset
solar.to_parquet(f"{out_dir}/solar.parquet", index=False)

print("Saved:", f"{out_dir}/solar.parquet")

Removed duplicates: 0
Saved: ../data/processed/weather/solar.parquet


In [24]:
# Step: Create PROCESSED version (no city filtering)

solar_processed = solar.copy()

# city_id mapping (only for project cities)
city_id_map = {
    "riyadh": 1,
    "jeddah": 2,
    "khobar": 3,
    "dammam": 4,
}

solar_processed["city_id"] = solar_processed["city_norm"].map(city_id_map)

# حفظ النسخة الكاملة
solar_processed.to_parquet(
    "../data/processed/weather/solar.parquet",
    index=False
)

print("Processed weather saved (no filtering)")
print("Total rows:", len(solar_processed))
print("Cities with ID:", solar_processed["city_id"].notna().sum())

Processed weather saved (no filtering)
Total rows: 6588
Cities with ID: 6588


In [25]:
# Step: Create FINAL version (filtered to project cities)

solar_final = solar_processed[solar_processed["city_id"].notna()].copy()

solar_final["city_id"] = solar_final["city_id"].astype("int64")

solar_final.to_parquet(
    "../data/final/weather_selected.parquet",
    index=False
)

print("Final weather saved (filtered)")
print("Rows:", len(solar_final))
print("Cities:", solar_final["city_id"].unique())

Final weather saved (filtered)
Rows: 6588
Cities: [1 2 4]


In [12]:
import pandas as pd

hist = pd.read_csv(
    "../data/raw/historical_sites/historical site list CSV.csv"
)

print("Shape:", hist.shape)
print("\nCOLUMNS:")
for c in hist.columns:
    print("-", c)

hist.head()



hist_clean = hist.rename(columns={
    "اسم الموقع باللغة العربية": "site_name_ar",
    "الوصف بالعربي": "description_ar"
})

hist_clean.head()


Shape: (40, 2)

COLUMNS:
- اسم الموقع باللغة العربية
- الوصف بالعربي


,site_name_ar,description_ar
0,أسبلة الملك عبدالعزيز على طريق الحج القديم,اهتم الملك المؤسس الملك عبد العزيز -طيب الله ث...
1,بئر طُوَى,"من الآبار التاريخية بوادي ""ذي طوى"" أحدِ أودية ..."
2,جبل الرحمة,جبل صغير بالنسبة لما حوله من الجبال، لا يزيد ا...
3,جبل النور وغار حِراء,من أهم الأماكن في تاريخ الإسلام وسيرة نبيه محم...
4,جبل وغار ثَور,"يقع ""جبل ثور"" على بعد أربعة كيلو مترات جنوبي ا..."


In [14]:
def normalize_text(series):
    s = series.astype(str)

    s = s.str.replace("\u200f", "", regex=False)
    s = s.str.replace("\u200e", "", regex=False)

    s = s.str.strip().str.lower()
    s = s.str.replace(r"\s+", " ", regex=True)

    return s

hist_norm = hist_clean.copy()

hist_norm["site_name_norm"] = normalize_text(hist_norm["site_name_ar"])

hist_norm.head()

city_aliases = pd.read_csv("../data/processed/geo/city_aliases.csv")

city_aliases.head()


,alias,city_geo,city_id,alias_norm
0,الرياض,riyadh,1,الرياض
1,riyadh,riyadh,1,riyadh
2,ar riyadh,riyadh,1,ar riyadh
3,al riyadh,riyadh,1,al riyadh
4,جدة,jeddah,2,جدة


In [15]:
def match_city(text, aliases_df):
    if pd.isna(text):
        return None, None

    for _, row in aliases_df.iterrows():
        if row["alias_norm"] in text:
            return row["city_id"], row["city_geo"]

    return None, None


hist_norm[["city_id", "city_geo"]] = hist_norm.apply(
    lambda r: pd.Series(
        match_city(
            r["site_name_norm"] + " " + normalize_text(pd.Series([r["description_ar"]]))[0],
            city_aliases
        )
    ),
    axis=1
)

hist_norm.head()


,site_name_ar,description_ar,site_name_norm,city_id,city_geo
0,أسبلة الملك عبدالعزيز على طريق الحج القديم,اهتم الملك المؤسس الملك عبد العزيز -طيب الله ث...,أسبلة الملك عبدالعزيز على طريق الحج القديم,None,None
1,بئر طُوَى,"من الآبار التاريخية بوادي ""ذي طوى"" أحدِ أودية ...",بئر طُوَى,None,None
2,جبل الرحمة,جبل صغير بالنسبة لما حوله من الجبال، لا يزيد ا...,جبل الرحمة,None,None
3,جبل النور وغار حِراء,من أهم الأماكن في تاريخ الإسلام وسيرة نبيه محم...,جبل النور وغار حِراء,None,None
4,جبل وغار ثَور,"يقع ""جبل ثور"" على بعد أربعة كيلو مترات جنوبي ا...",جبل وغار ثَور,None,None


In [17]:
out_path = "../data/processed/poi/historical_sites.parquet"

hist_norm.to_parquet(out_path, index=False)

print("Saved:", out_path)


Saved: ../data/processed/poi/historical_sites.parquet


In [53]:
import pandas as pd

df = pd.read_csv("../data/raw/xmap_poi/Retail_and_Shopping_data.csv")
print(df.shape)
print(df.columns)


(1000, 19)
Index(['poi_id', 'name', 'region', 'locality', 'address', 'postcode',
       'longitude', 'latitude', 'category_level1', 'category_level2',
       'category_level3', 'business_status', 'phone', 'website_domain',
       'rating', 'rating_count', 'traffic_score', 'time_spent',
       'update_month'],
      dtype='object')


In [60]:
import pandas as pd

# 1) مسار الملف (عدليه إذا اسم ملفك مختلف)
path = "../data/raw/xmap_poi/Travel_and_Lodging_data.csv"

df = pd.read_csv(path)

print("Shape:", df.shape)
print("\nColumns:")
print(df.columns.tolist())

df.head()


Shape: (1000, 19)

Columns:
['poi_id', 'name', 'region', 'locality', 'address', 'postcode', 'longitude', 'latitude', 'category_level1', 'category_level2', 'category_level3', 'business_status', 'phone', 'website_domain', 'rating', 'rating_count', 'traffic_score', 'time_spent', 'update_month']


,poi_id,name,region,locality,address,postcode,longitude,latitude,category_level1,category_level2,category_level3,business_status,phone,website_domain,rating,rating_count,traffic_score,time_spent,update_month
0,45d4a0aae77ad011165983d71a2a8fe7,Alfursan Lounge - jeddah airport,Jeddah,Jeddah,"طريق المدينة المنورة،،, King Abdulaziz Interna...",23635,39.171409,21.662734,Travel & Lodging,Travel Agencies,Travel lounge,Open 24 hours,+966 12 699 9996,saudia.com,4.3,20574.0,64.43,People typically spend up to 1.5 hours here,03-2025
1,0b5f77bbe9c7b09a366f941317699e68,Al Sheraa Camp,Makkah Province,Rabigh,"Ankas Near Petrorabigh, Rabigh 21911, Saudi Ar...",21911,39.035451,22.748565,Travel & Lodging,Vacation Rentals & RV,Camping cabin,Open,+966 12 422 1136,gsest.sa,3.0,3.0,33.00,People typically spend up to 4 hours here,03-2025
2,66a13e4d25016031ccc54edac4c2f31d,Akbar Travels And Tourism Co. Al Khobar,Dammam Metropolitan Area,Al Khobar,"King Abdulaziz St, Al Khobar Al Shamalia, Al K...",34429,50.204298,26.287323,Travel & Lodging,Travel Agencies,Travel agency,Open,+966 13 804 0777,akbartravels.com,4.7,99.0,39.43,People typically spend up to 2.5 hours here,03-2025
3,56c990fdd1b320823d2ff1f218fc2c37,Sultan middle east immigration,Riyadh Province,Riyadh,"2551 Mousa Complex, Al Olaya, Building 4, grou...",12221,46.687292,24.685396,Travel & Lodging,Travel Agencies,Travel agency,Open,+966 59 682 7467,sultanimmigration.com,4.5,64.0,65.67,People typically spend up to 3 hours here,03-2025
4,74766ff789833fd4c72bae688f30a526,مكتب الفارس حجزتذاكر الباصات اليمن,Al Madinah Province,Madinah,"حي المستراح، شارع عبد المجيد بن عبد العزيز, Ma...",42313,39.603139,24.485861,Travel & Lodging,Travel Agencies,Travel agency,Open 24 hours,+966 50 746 1473,maps.app.goo.gl,4.7,44.0,43.00,People typically spend up to 2.5 hours here,03-2025


In [61]:
# 2) اختيار الأعمدة المطلوبة
selected_cols = [
    "poi_id",
    "name",
    "region",
    "locality",
    "address",
    "longitude",
    "latitude",
    "category_level1",
    "category_level2",
    "category_level3",
    "business_status",
    "phone",
    "website_domain",
    "rating",
    "rating_count",
    "traffic_score",
    "time_spent"
]

df_sel = df[selected_cols].copy()

print("Shape after selecting columns:", df_sel.shape)
df_sel.head()


Shape after selecting columns: (1000, 17)


,poi_id,name,region,locality,address,longitude,latitude,category_level1,category_level2,category_level3,business_status,phone,website_domain,rating,rating_count,traffic_score,time_spent
0,45d4a0aae77ad011165983d71a2a8fe7,Alfursan Lounge - jeddah airport,Jeddah,Jeddah,"طريق المدينة المنورة،،, King Abdulaziz Interna...",39.171409,21.662734,Travel & Lodging,Travel Agencies,Travel lounge,Open 24 hours,+966 12 699 9996,saudia.com,4.3,20574.0,64.43,People typically spend up to 1.5 hours here
1,0b5f77bbe9c7b09a366f941317699e68,Al Sheraa Camp,Makkah Province,Rabigh,"Ankas Near Petrorabigh, Rabigh 21911, Saudi Ar...",39.035451,22.748565,Travel & Lodging,Vacation Rentals & RV,Camping cabin,Open,+966 12 422 1136,gsest.sa,3.0,3.0,33.00,People typically spend up to 4 hours here
2,66a13e4d25016031ccc54edac4c2f31d,Akbar Travels And Tourism Co. Al Khobar,Dammam Metropolitan Area,Al Khobar,"King Abdulaziz St, Al Khobar Al Shamalia, Al K...",50.204298,26.287323,Travel & Lodging,Travel Agencies,Travel agency,Open,+966 13 804 0777,akbartravels.com,4.7,99.0,39.43,People typically spend up to 2.5 hours here
3,56c990fdd1b320823d2ff1f218fc2c37,Sultan middle east immigration,Riyadh Province,Riyadh,"2551 Mousa Complex, Al Olaya, Building 4, grou...",46.687292,24.685396,Travel & Lodging,Travel Agencies,Travel agency,Open,+966 59 682 7467,sultanimmigration.com,4.5,64.0,65.67,People typically spend up to 3 hours here
4,74766ff789833fd4c72bae688f30a526,مكتب الفارس حجزتذاكر الباصات اليمن,Al Madinah Province,Madinah,"حي المستراح، شارع عبد المجيد بن عبد العزيز, Ma...",39.603139,24.485861,Travel & Lodging,Travel Agencies,Travel agency,Open 24 hours,+966 50 746 1473,maps.app.goo.gl,4.7,44.0,43.00,People typically spend up to 2.5 hours here


In [62]:
# 3) فحص القيم الفاضية
missing = df_sel.isna().sum().sort_values(ascending=False)
missing


time_spent         867
traffic_score      273
website_domain     110
phone               10
poi_id               0
name                 0
region               0
locality             0
address              0
category_level2      0
category_level1      0
latitude             0
longitude            0
category_level3      0
business_status      0
rating_count         0
rating               0
dtype: int64

In [63]:
# 4) التأكد من وجود إحداثيات
df_sel[df_sel['latitude'].isna() | df_sel['longitude'].isna()].shape


(0, 17)

In [64]:
# 5) حذف التكرار
df_clean = df_sel.drop_duplicates(subset=["poi_id"])

print("Before:", df_sel.shape)
print("After :", df_clean.shape)


Before: (1000, 17)
After : (1000, 17)


In [65]:
out_path = "../data/processed/poi/travel_and_lodging_clean.parquet"
df_clean.to_parquet(out_path, index=False)

print("Saved to:", out_path)


Saved to: ../data/processed/poi/travel_and_lodging_clean.parquet


In [28]:
import pandas as pd

events_raw = pd.read_csv("../data/raw/events/events_data.csv")

print(events_raw.shape)
events_raw.head()

(5701, 14)


,Name,City,StartDate,StartDateFormatted,EndDate,EndDateFormatted,StartTime,StartTimeFormatted,EndTime,EndTimeFormatted,IsMaleAllowed,IsFemaleAllowed,IsFamilyAllowed,EventMode
0,الألعاب النارية,NaN,2018-09-23T00:00:00Z,23-09-2018,2018-09-23T00:00:00Z,23-09-2018,22:00:00,10:00 م,22:30:00,10:30 م,True,True,True,IsExpired
1,الألعاب النارية,NaN,2018-09-23T00:00:00Z,23-09-2018,2018-09-23T00:00:00Z,23-09-2018,22:00:00,10:00 م,22:30:00,10:30 م,True,True,True,IsExpired
2,الألعاب النارية,NaN,2018-09-23T00:00:00Z,23-09-2018,2018-09-23T00:00:00Z,23-09-2018,22:00:00,10:00 م,22:30:00,10:30 م,True,True,True,IsExpired
3,الألعاب النارية,NaN,2018-09-23T00:00:00Z,23-09-2018,2018-09-23T00:00:00Z,23-09-2018,22:00:00,10:00 م,22:30:00,10:30 م,True,True,True,IsExpired
4,غوفالي,جدة,2024-11-15T00:00:00,15-11-2024,2026-02-14T00:00:00,14-02-2026,21:00:00,9:00 م,03:00:00,3:00 ص,True,True,True,IsActive


In [29]:
print(events_raw.columns.tolist())

['Name', 'City', 'StartDate', 'StartDateFormatted', 'EndDate', 'EndDateFormatted', 'StartTime', 'StartTimeFormatted', 'EndTime', 'EndTimeFormatted', 'IsMaleAllowed', 'IsFemaleAllowed', 'IsFamilyAllowed', 'EventMode']


In [30]:
events = events_raw[
    [
        "Name",
        "City",
        "StartDate",
        "EndDate",
        "StartTime",
        "EndTime",
        "EventMode",
        "IsMaleAllowed",
        "IsFemaleAllowed",
        "IsFamilyAllowed"
    ]
].copy()

events.shape

(5701, 10)

In [32]:
events = events_raw[
    [
        "Name",
        "City",
        "StartDate",
        "EndDate",
        "StartTime",
        "EndTime",
        "EventMode",
        "IsMaleAllowed",
        "IsFemaleAllowed",
        "IsFamilyAllowed"
    ]
].copy()

events.shape

(5701, 10)

In [33]:
events = events.drop_duplicates(
    subset=["Name", "City", "StartDate"]
)

events.shape

(5622, 10)

In [34]:
events = events.reset_index(drop=True)
events["event_id"] = events.index + 1

events.head()

,Name,City,StartDate,EndDate,StartTime,EndTime,EventMode,IsMaleAllowed,IsFemaleAllowed,IsFamilyAllowed,event_id
0,الألعاب النارية,NaN,2018-09-23T00:00:00Z,2018-09-23T00:00:00Z,22:00:00,22:30:00,IsExpired,True,True,True,1
1,غوفالي,جدة,2024-11-15T00:00:00,2026-02-14T00:00:00,21:00:00,03:00:00,IsActive,True,True,True,2
2,مدينة بيلي بيز,جدة,2022-03-13T00:00:00,2026-03-13T00:00:00,09:00:00,01:00:00,IsActive,True,True,True,3
3,اكستريم بلاي,جدة,2025-07-23T00:00:00,2030-07-23T00:00:00,15:00:00,00:00:00,IsActive,True,True,True,4
4,بوست,جدة,2023-01-02T00:00:00,2031-01-02T00:00:00,14:00:00,01:00:00,IsActive,True,True,True,5


In [35]:
event_modes = (
    events[["EventMode"]]
    .dropna()
    .drop_duplicates()
    .reset_index(drop=True)
)

event_modes["mode_id"] = range(1, len(event_modes) + 1)

event_modes

,EventMode,mode_id
0,IsExpired,1
1,IsActive,2


In [36]:
event_modes.to_parquet(
    "../data/final/event_modes.parquet",
    index=False
)

In [37]:
events = events.merge(
    event_modes,
    on="EventMode",
    how="left"
)

events = events.drop(columns=["EventMode"])

events.head()

,Name,City,StartDate,EndDate,StartTime,EndTime,IsMaleAllowed,IsFemaleAllowed,IsFamilyAllowed,event_id,mode_id
0,الألعاب النارية,NaN,2018-09-23T00:00:00Z,2018-09-23T00:00:00Z,22:00:00,22:30:00,True,True,True,1,1
1,غوفالي,جدة,2024-11-15T00:00:00,2026-02-14T00:00:00,21:00:00,03:00:00,True,True,True,2,2
2,مدينة بيلي بيز,جدة,2022-03-13T00:00:00,2026-03-13T00:00:00,09:00:00,01:00:00,True,True,True,3,2
3,اكستريم بلاي,جدة,2025-07-23T00:00:00,2030-07-23T00:00:00,15:00:00,00:00:00,True,True,True,4,2
4,بوست,جدة,2023-01-02T00:00:00,2031-01-02T00:00:00,14:00:00,01:00:00,True,True,True,5,2


In [38]:
city_aliases = pd.read_csv("../data/processed/geo/city_aliases.csv")

events["city_norm"] = (
    events["City"]
    .astype(str)
    .str.strip()
    .str.lower()
)

In [39]:
events = events.merge(
    city_aliases[["alias_norm", "city_id"]],
    left_on="city_norm",
    right_on="alias_norm",
    how="left"
)

events.head()

,Name,City,StartDate,EndDate,StartTime,EndTime,IsMaleAllowed,IsFemaleAllowed,IsFamilyAllowed,event_id,mode_id,city_norm,alias_norm,city_id
0,الألعاب النارية,NaN,2018-09-23T00:00:00Z,2018-09-23T00:00:00Z,22:00:00,22:30:00,True,True,True,1,1,nan,NaN,NaN
1,غوفالي,جدة,2024-11-15T00:00:00,2026-02-14T00:00:00,21:00:00,03:00:00,True,True,True,2,2,جدة,جدة,2.0
2,مدينة بيلي بيز,جدة,2022-03-13T00:00:00,2026-03-13T00:00:00,09:00:00,01:00:00,True,True,True,3,2,جدة,جدة,2.0
3,اكستريم بلاي,جدة,2025-07-23T00:00:00,2030-07-23T00:00:00,15:00:00,00:00:00,True,True,True,4,2,جدة,جدة,2.0
4,بوست,جدة,2023-01-02T00:00:00,2031-01-02T00:00:00,14:00:00,01:00:00,True,True,True,5,2,جدة,جدة,2.0


In [40]:
events = events.drop(
    columns=["City", "city_norm", "alias_norm"]
)

In [41]:
events.to_parquet(
    "../data/processed/events/events_clean.parquet",
    index=False
)

In [ ]:
events.to_parquet(
    "../data/processed/events/events_clean.parquet",
    index=False
)

print("Events CLEAN saved in processed ✅")

In [24]:
events_final = events[events["city_id"].isin([1,2,3,4])]

events_final.shape

(4270, 11)

In [25]:
events_final.to_parquet(
    "../data/final/events_selected.parquet",
    index=False
)

print("Events FINAL saved ✅")

Events FINAL saved ✅
